In [1]:
from xml.etree import ElementTree as ET
import folium
from shapely.geometry import Point, LineString

Caminhos para os arquivos KML

In [20]:
planned_trip_path = '/home/marcelo-borges/Documentos/Projetos/comparison_route/dados/911 V2.kml'
trip_made_path = '/home/marcelo-borges/Documentos/Projetos/comparison_route/dados/20224.kml'

Namespace para o arquivo KML

In [22]:

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

Função para ler e extrair Placemarks de um arquivo KML

In [2]:

def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks


Função para converter string de coordenadas em lista de tuplas (latitude, longitude)

In [3]:

def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

Função para adicionar Placemarks ao mapa

In [4]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)
              
        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                            folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)
            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)


Extrair os Placemarks de cada arquivo

In [7]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)
placemarks_trip_made = read_and_extract_placemarks(trip_made_path, ns)

Determinar o centro do mapa a partir das coordenadas do primeiro arquivo

In [8]:

first_coords = parse_coordinates(placemarks_planned[0]['coordinates'])
map_center = first_coords[len(first_coords) // 2]

tratando as LineStrings

In [9]:
planned_path_coords = parse_coordinates(placemarks_planned[0]['coordinates'])
planned_line = LineString(planned_path_coords)

Criar o mapa

In [10]:
map = folium.Map(location=map_center, zoom_start=12)

Adicionar os Placemarks ao mapa

In [11]:
add_placemarks_to_map(placemarks_planned, map)
add_placemarks_to_map(placemarks_trip_made, map)

distancia limite para gerar ocorrência

In [12]:
distance_ocorr = 100

criando a relação de coordenadas que está a {distance_ocorr} do planejado

In [13]:
registered_path_coords = [parse_coordinates(placemark['coordinates'])[0] for placemark in placemarks_trip_made]
significant_deviations = []
meters_to_degrees = distance_ocorr / 111000 

Lendo as coordenadas das ocorrências e criando um dicionário contendo as coordenadas e um informação para ser posta no tooltip, no caso informação de distância

In [14]:
for point_coords in registered_path_coords:
    point = Point(point_coords)
    distance = planned_line.distance(point) * 111000
    if distance > distance_ocorr:
        significant_deviations.append({'location': point_coords, 'info': f'Desvio de {distance:.2f} metros'})


Adicionar pontos de desvio significativo no mapa

In [15]:
for deviation in significant_deviations:
    folium.Marker(
        location=deviation['location'],
        icon=folium.Icon(color='red', icon='info-sign'),
        tooltip=deviation['info']
    ).add_to(map)


Criando legenda e plotando no mapa

In [16]:
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 400px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">&nbsp; <b>Legenda do Mapa</b> <br>
     &nbsp; Vermelho : Rota Planejada sentido ida <br>
     &nbsp; Verde : Rota Planejamento sentido volta <br>
     &nbsp; Símbolo azul : Posição coletada do ônibus <br>
     &nbsp; Símbolo vermelho : Desvios Significativos <br>
      </div> 
     '''
map.get_root().html.add_child(folium.Element(legend_html))

In [17]:
map

Salvando o mapa

In [18]:
map.save('/home/marcelo-borges/Documentos/Projetos/comparison_route/dados/map.html')

A partir daqui são teste e não são necessários para o funcionamento da engine

In [19]:
placemarks_planned

[{'name': '911-1i - T. PRACA A / AV. T-2 / SETOR BUENO - IDA',
  'coordinates': '-49.28469100000001,-16.673701,0 -49.284677637,-16.673687237,0 -49.28468503800001,-16.673677144,0 -49.28475580200001,-16.673715205,0 -49.284751278,-16.673735129,0 -49.284730533,-16.673762111,0 -49.284692354,-16.673774961,0 -49.284634728,-16.673792308,0 -49.284613292,-16.673823464,0 -49.284605937,-16.673868109,0 -49.284615038,-16.673903487,0 -49.28463755,-16.673937579,0 -49.28468199999999,-16.674011,0 -49.28468199999999,-16.674011,0 -49.28504483899999,-16.675592307,0 -49.28504483899999,-16.675592307,0 -49.285484791,-16.677640164,0 -49.285550913,-16.677918357,0 -49.285567702,-16.677962462,0 -49.285587831,-16.677994792,0 -49.285614666,-16.678027122,0 -49.285643525,-16.678056131,0 -49.285697866,-16.678076148,0 -49.285822691,-16.678094228,0 -49.285954893,-16.678127725,0 -49.286040439,-16.678169846,0 -49.286115928,-16.678217106,0 -49.286178391,-16.678272442,0 -49.286221693,-16.678319381,0 -49.286256949,-16.678367